In [1]:
import math, random, re
from collections import defaultdict, Counter
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
import requests

In [2]:
def plot_resumes(plt):
    data = [ ("big data", 100, 15), ("Hadoop", 95, 25), ("Python", 75, 50),
         ("R", 50, 40), ("machine learning", 80, 20), ("statistics", 20, 60),
         ("data science", 60, 70), ("analytics", 90, 3),
         ("team player", 85, 85), ("dynamic", 2, 90), ("synergies", 70, 0),
         ("actionable insights", 40, 30), ("think out of the box", 45, 10),
         ("self-starter", 30, 50), ("customer focus", 65, 15),
         ("thought leadership", 35, 35)]

    def text_size(total):
        """equals 8 if total is 0, 28 if total is 200"""
        return 8 + total / 200 * 20

    for word, job_popularity, resume_popularity in data:
        plt.text(job_popularity, resume_popularity, word,
                 ha='center', va='center',
                 size=text_size(job_popularity + resume_popularity))
    plt.xlabel("Popularity on Job Postings")
    plt.ylabel("Popularity on Resumes")
    plt.axis([0, 100, 0, 100])
    plt.show()


In [3]:
#
# n-gram models
#

def fix_unicode(text):
    return text.replace(u"\u2019", "'")

def get_document():

    url = "http://radar.oreilly.com/2010/06/what-is-data-science.html"
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html5lib')

    content = soup.find("div", "article-body")         # find article-body div
    regex = r"[\w']+|[\.]"                             # matches a word or a period

    document = []


    for paragraph in content("p"):
        words = re.findall(regex, fix_unicode(paragraph.text))
        document.extend(words)

    return document

def generate_using_bigrams(transitions):
    current = "."   # this means the next word will start a sentence
    result = []
    while True:
        next_word_candidates = transitions[current]    # bigrams (current, _)
        current = random.choice(next_word_candidates)  # choose one at random
        result.append(current)                         # append it to results
        if current == ".": return " ".join(result)     # if "." we're done

def generate_using_trigrams(starts, trigram_transitions):
    current = random.choice(starts)   # choose a random starting word
    prev = "."                        # and precede it with a '.'
    result = [current]
    while True:
        next_word_candidates = trigram_transitions[(prev, current)]
        next = random.choice(next_word_candidates)

        prev, current = current, next
        result.append(current)

        if current == ".":
            return " ".join(result)


In [5]:
def is_terminal(token):
    return token[0] != "_"

def expand(grammar, tokens):
    for i, token in enumerate(tokens):

        # ignore terminals
        if is_terminal(token): continue

        # choose a replacement at random
        replacement = random.choice(grammar[token])

        if is_terminal(replacement):
            tokens[i] = replacement
        else:
            tokens = tokens[:i] + replacement.split() + tokens[(i+1):]
        return expand(grammar, tokens)

    # if we get here we had all terminals and are done
    return tokens

def generate_sentence(grammar):
    return expand(grammar, ["_S"])


In [31]:
#
# Gibbs Sampling
#

def roll_a_die():
    return random.choice([1,2,3,4,5,6])

def direct_sample():
    d1 = roll_a_die()
    d2 = roll_a_die()
    return d1, d1 + d2

def random_y_given_x(x):
    """equally likely to be x + 1, x + 2, ... , x + 6"""
    return x + roll_a_die()

def random_x_given_y(y):
    if y <= 7:
        # if the total is 7 or less, the first die is equally likely to be
        # 1, 2, ..., (total - 1)
        return random.randrange(1, y)
    else:
        # if the total is 7 or more, the first die is equally likely to be
        # (total - 6), (total - 5), ..., 6
        return random.randrange(y - 6, 7)

def gibbs_sample(num_iters=100):
    x, y = 1, 2 # doesn't really matter
    for _ in range(num_iters):
        x = random_x_given_y(y)
        y = random_y_given_x(x)
    return x, y

def compare_distributions(num_samples=1000):
    counts = defaultdict(lambda: [0, 0])
    for _ in range(num_samples):
        counts[gibbs_sample()][0] += 1
        counts[direct_sample()][1] += 1
    return counts


In [65]:
#
# TOPIC MODELING
#

def sample_from(weights):
    total = sum(weights)
    rnd = total * random.random()       # uniform between 0 and total
    print(total, rnd)
    for i, w in enumerate(weights):
        rnd -= w                        # return the smallest i such that
        if rnd <= 0: return i           # sum(weights[:(i+1)]) >= rnd

documents = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

K = 4

document_topic_counts = [Counter()
                         for _ in documents]

topic_word_counts = [Counter() for _ in range(K)]

topic_counts = [0 for _ in range(K)]

document_lengths = [len(d) for d in documents]

distinct_words = set(word for document in documents for word in document)
W = len(distinct_words)

D = len(documents)

def p_topic_given_document(topic, d, alpha=0.1):
    """the fraction of words in document _d_
    that are assigned to _topic_ (plus some smoothing)"""

    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    """the fraction of words assigned to _topic_
    that equal _word_ (plus some smoothing)"""

    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + W * beta))

def topic_weight(d, word, k):
    """given a document and a word in that document,
    return the weight for the k-th topic"""

    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k)
                        for k in range(K)])

random.seed(0)
document_topics = [[random.randrange(K) for word in document]
                   for document in documents]

for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

for iter in range(1000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):

            # remove this word / topic from the counts
            # so that it doesn't influence the weights
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1

            # choose a new topic based on the weights
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic

            # and now add it back to the counts
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1


0.013724578130869624 0.0009481658905078121
0.039503415909913284 0.026811914852602214
0.019699100936164518 0.002565304594845868
0.02762817982578488 0.004131803494463397
0.004931551942569945 0.0001905629136651609
0.0047522339639525235 0.0003813587816553901
0.005554583449412838 0.0038844468742036827
0.0048345752687551724 0.00400960751052785
0.03049439931567599 0.02084287269856107
0.025057995915904633 0.020995223542127465
0.061810829513317095 0.03242435031758614
0.018735256712134545 0.009775871486853578
0.037381779654155366 0.008803473976449106
0.013892392436952623 0.0029896608448898264
0.004577712297242479 0.0031104381251227877
0.004193489940335175 0.0034607333411158414
0.021400962900020522 0.00897606123635564
0.021400962900020522 0.005889776463859791
0.05802258950905112 0.028585141259125108
0.05802258950905112 0.037203900147248864
0.04159605747183127 0.02912662113894337
0.07129700587443111 0.07004648387719196
0.006887274252072934 0.002461288297954639
0.016907469584961563 0.00548351242514

0.021638626210718208 0.0215769497069369
0.008535313942223018 5.496097177445117e-05
0.10469962838936746 0.05606917545241158
0.03830951506760112 0.016406801600608784
0.07461505290773718 0.003713617756029422
0.07461505290773718 0.028704007631747547
0.005367547200664148 0.0014625751193516024
0.105563050621974 0.10251749698657449
0.05546529891131908 0.031319421271733656
0.005367547200664148 0.0012321886136584166
0.05546529891131908 0.039743618901412696
0.027275559559541755 0.014909843764124882
0.009218241154224377 0.002601084314742681
0.004737954415873122 0.003505468336243447
0.04167928666939327 0.03757674206646481
0.0785716477805044 0.005090435730610333
0.00625534415493237 0.0019236337386278374
0.04724085455452346 0.015462374426656275
0.004479775973114188 0.0016718385931036098
0.004479775973114188 0.002151500540034208
0.00536756006402328 0.0031242441468762204
0.005656539545974708 0.0007765747216890016
0.05860075442200776 0.0009082391155370092
0.006619559576791349 0.003341882696678692
0.059

0.04839659210211595 0.025866623142516452
0.00932177975727114 0.007947400918373246
0.00932177975727114 0.0013442681028698886
0.06496586267430549 0.025494513811973922
0.05582951879219889 0.04286279307329347
0.10262918031788781 0.07502218368283792
0.007575299117533152 0.007397204014669372
0.045399589963408815 0.019656053042134497
0.006128962118137084 0.0006424301588153059
0.04122254500583762 0.01899066452944228
0.058206657917311846 0.026890753382191723
0.04119816196616357 0.006987570369955173
0.034676919941766894 0.01644931143707274
0.019485947719544674 0.0030984472861295344
0.004194312238299499 0.0038521076910312748
0.004294975497322453 0.0037133983638705905
0.058687334137294 0.04470982501527208
0.10924588575516765 0.012291871866734829
0.005636260086718455 0.0020757358266040526
0.005636260086718455 0.0008042568076667922
0.0059524067677943545 0.002128707419628498
0.03170064497258241 0.014984207949851678
0.004371129095435088 0.0022673801935285346
0.005552960724128044 0.00029933369525678535

0.09815521820555538 0.01634113402047317
0.00848142825950151 0.0023349838643245825
0.10268535204746605 0.020681716684806027
0.03689587836325552 0.020465134482770247
0.01640512686467121 0.014191548566039354
0.021652643146755596 0.0019251034657926872
0.021652643146755596 0.010885414274538452
0.017986404537030477 0.014955408000246669
0.03559517590458216 0.004367818444153197
0.09900097300603143 0.05876406594449314
0.09900097300603143 0.04583334710142149
0.007169215378048765 0.0023237621069532944
0.14420130093954608 0.04276533572119381
0.07568525815879742 0.047554876953000785
0.007169215378048765 0.004829407346451022
0.07568525815879742 0.021701408814004217
0.05533116080755514 0.0012673762235336874
0.10504706989846424 0.0008635846801780207
0.005615251716646043 0.00474859859504421
0.03157823175498827 0.0024054319106203296
0.057619898421654936 0.056227784490707394
0.05048519522530791 0.015807599839121677
0.03758784713960366 0.03674992432782618
0.005304313879507898 0.0034663256769172012
0.00533

0.052657983283081064 0.013579400240073543
0.06703064272980917 0.05285235220841547
0.02805832835152308 0.0040759958115579534
0.004631286852192424 0.0028538477245650365
0.0041691498002177614 0.0008523260367873687
0.02543107214473134 0.0034675645225778207
0.005818371372610279 0.004907299939998088
0.10897950114602106 0.04434983203540355
0.10897950114602106 0.1037766740279637
0.07394874243105183 0.014706550208522908
0.006625450630905253 0.0039009880169938557
0.032828913984490476 0.011537678534509422
0.007387118906157834 0.004864347988639288
0.1092249515200358 0.0740657771606349
0.005615325851586608 0.004241593195994734
0.05742013868581121 0.042565382176388375
0.11931637180919044 0.009731828213919897
0.11931637180919044 0.0859356398078892
0.04364784403957542 0.026972126308997302
0.005039177257309676 0.0034833628312545524
0.004140220047672854 0.0011562340671165926
0.004140220047672854 0.0024805160745519457
0.007732676048317184 0.001705626472507079
0.07528026569115881 0.06731903707150931
0.005

...

0.07247577577753878 0.05225682091546411
0.07247577577753878 0.058799679096052056
0.003922038493151339 0.0007870589249032136
0.003922038493151339 0.0018630337310238166
0.06085194685522193 0.037715348948584936
0.06085194685522193 0.03732066094631114
0.005926362163513913 0.0035121013900083413
0.0074269624036099515 0.006251679255059768
0.007023939748764881 0.00023454976391719151
0.04071485313492986 0.010169763340983622
0.0758084360226304 0.005866032674350291
0.04071485313492986 0.020256016396072413
0.05742608308145393 0.01645703609122926
0.10923089591567853 0.09260460099282664
0.0056212702472293275 0.004645545204510608
0.10923089591567853 0.03483012965820731
0.06977644655382348 0.0582091440413411
0.10267118339592875 0.024470149867596342
0.036881709711718204 0.015551620427469548
0.0170339435357412 0.000329773813926252
0.022792664496628694 0.008467975745348326
0.022792664496628694 0.01513137341879742
0.02919670337651962 0.021015530530218197
0.036881709711718204 0.012126378458906161
0.1026711

0.008433549617828367 0.004166765999162157
0.005606010729118154 0.0009158828030351128
0.10921563639756734 0.0442422088008041
0.0943745823461515 0.05462610697677343
0.1390174394890087 0.05460954283084492
0.007329011429612852 0.003822972373062044
0.00703376629736487 0.006005518051549724
0.05741082356334276 0.016740059015257734
0.05741082356334276 0.025364273356258966
0.008461521865611587 0.0007701732963490668
0.007125026618185979 0.0061435368160174236
0.07524322806934793 0.06969287060838836
0.05362441105101517 0.0015556395113234364
0.0044785035712209854 0.0044248755167540735
0.06995052577852665 0.06822095395210989
0.037332680660681544 0.02007389716386649
0.004714835542836425 0.004391975184014271
0.027934491650842046 0.02379015056221028
0.0473552100018524 0.0406142613610301
0.008924789289878287 0.008238402336641736
0.004879481846512591 0.0007310235567335883
0.03448780288359479 0.0011283258109684548
0.04715348444825447 0.03688685241927021
0.05015812305316195 0.048693379192396094
0.042503469

0.005342402519821874 0.003977352650849484
0.05649234278266536 0.03993074331718379
0.05544015423047681 0.035114593261000325
0.005342402519821874 0.0009631973577909648
0.05544015423047681 0.025528480144382113
0.05235712863401037 0.04423117233493387
0.05428614097968938 0.052403778166887544
0.005049105836972189 0.0037069064161817833
0.043452645353642995 0.03861918127773549
0.08213589778082743 0.04651027261101446
0.00643010487613551 0.0005629059919478345
0.044320151035589464 0.030695874237790497
0.004945526926989071 0.004704766622117851
0.004945526926989071 0.002926317948362829
0.014186118324838534 0.008229042319324816
0.005342854529681909 0.0051663277802006646
0.04541425110847748 0.005568419129463629
0.04541425110847748 0.010281986484580426
0.08544524284401467 0.07566937618096366
0.04541425110847748 0.014644327524322867
0.08438979600867294 0.05478569261603363
0.014436060336153368 0.01102435362761699
0.004547531030148333 0.0023876696731356638
0.004547531030148333 0.001320864429085757
0.0381

0.005459948930458161 0.0011283579371431296
0.005415183653077044 0.0052624414994966165
0.02220102035902959 0.004850767096831063
0.005095564104616492 0.0007066920383638014
0.05263730807493189 0.01060920600540439
0.05263730807493189 0.052214622235345655
0.10017905204524725 0.09203755063868287
0.05263730807493189 0.02223876521036109
0.12983728609572445 0.05074760924985055
0.04625813172010992 0.04147646977861893
0.004468554532302645 0.0015033007598884676
0.004468554532302645 0.0030874142149818194
0.04625813172010992 0.015606004797815315
0.04625813172010992 0.024382747644815614
0.03812611386232053 0.03209641285229925
0.03812611386232053 0.011481344177981008
0.026154469580971156 0.0031517796798300243
0.03313842715807277 0.012088169530398232
0.03313842715807277 0.02812419937968322
0.023990738223041312 0.01863246181874055
0.04237309116421779 0.0011309479271610365
0.00553044717095995 0.004078514778260745
0.02502017077124344 0.010934702899246083
0.10529403506487962 0.07913022396680557
0.105294035

0.014900497455882272 0.01322278425531002
0.09671177489688246 0.0010504427467404923
0.06958493870690396 0.04378699637512241
0.07712204445829543 0.02890180143591586
0.033638742046421595 0.014146874049965273
0.056666189829053706 0.02094526556080727
0.006803829285093625 0.0006127186726121295
0.07970804826254248 0.04519217436612038
0.005292739970550988 0.0012318655176367988
0.02544314055777725 0.025258129013276804
0.03256181764953695 0.029337682736533276
0.03256181764953695 0.02046028245996272
0.05002389343418875 0.029106704687983303
0.027927933838229155 0.007500924604819008
0.005743397973974415 0.003846205149450182
0.005831974242269558 0.0008283995378791854
0.10338705408728838 0.017244504056097815
0.10338705408728838 0.09000896521176589
0.005347838401013851 0.0002820143451765891
0.005616291894130704 0.004067638654195909
0.02869401288741303 0.0253534535735299
0.0056929147587195745 0.004663361213422241
0.004567174652118913 0.00412653319285131
0.01810370096110828 0.005018478480705411
0.041244

In [66]:
if __name__ == "__main__":

    document = get_document()

    bigrams = list(zip(document, document[1:]))
    transitions = defaultdict(list)
    for prev, current in bigrams:
        transitions[prev].append(current)

    random.seed(0)
    print("bigram sentences")
    for i in range(10):
        print(i, generate_using_bigrams(transitions))
    print()

    # trigrams

    trigrams = list(zip(document, document[1:], document[2:]))
    trigram_transitions = defaultdict(list)
    starts = []

    for prev, current, next in trigrams:

        if prev == ".":              # if the previous "word" was a period
            starts.append(current)   # then this is a start word

        trigram_transitions[(prev, current)].append(next)

    print("trigram sentences")
    for i in range(10):
        print(i, generate_using_trigrams(starts, trigram_transitions))
    print()

    grammar = {
        "_S"  : ["_NP _VP"],
        "_NP" : ["_N",
                 "_A _NP _P _A _N"],
        "_VP" : ["_V",
                 "_V _NP"],
        "_N"  : ["data science", "Python", "regression"],
        "_A"  : ["big", "linear", "logistic"],
        "_P"  : ["about", "near"],
        "_V"  : ["learns", "trains", "tests", "is"]
    }

    print("grammar sentences")
    for i in range(10):
        print(i, " ".join(generate_sentence(grammar)))
    print()

    print("gibbs sampling")
    comparison = compare_distributions()
    for roll, (gibbs, direct) in comparison.items():
        print(roll, gibbs, direct)


    # topic MODELING

    for k, word_counts in enumerate(topic_word_counts):
        for word, count in word_counts.most_common():
            if count > 0: print(k, word, count)

    topic_names = ["Big Data and programming languages",
                   "databases",
                   "machine learning",
                   "statistics"]

    for document, topic_counts in zip(documents, document_topic_counts):
        print(document)
        for topic, count in topic_counts.most_common():
            if count > 0:
                print(topic_names[topic], count)
        print()


bigram sentences
0 But that's going to be current to generate a large searches correlates what that can be saying you can you want to figure out what's important role in enabling agile practices are then combine entrepreneurship with hundreds of track titles artists album .
1 CDDB views music by Jeff Hammerbacher said on the Philadelphia County by analyzing musical problem isn't the relational database .
2 If anything from machine consumable .
3 It was probably generated by their path .
4 Roger Magoulas who runs the metadata track titles .
5 The result .
6 Amazon understands that nobody remembers says that nobody remembers says that gave them open source R is really necessary for working with gathering data collection tools like the ability to be nice if it necessary to do data scientists particularly physicists rather than for distributing an audio stream processing companies banks and other disciplines it arrives and are easier to use .
7 Point of them open source the low 1 The Turk 